<a href="https://colab.research.google.com/github/winny0217/gpt2/blob/main/gpt2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [2]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

# 選擇 GPT-2 中文模型（更符合需求）
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

if torch.cuda.is_available():
    model.to("cuda")

def predict_next_word(input_text, max_new_tokens=1, temperature=0.8, top_k=50):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    # 使用 generate 生成
    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,   # 每次只生成一個 token
        do_sample=True,                  # 採樣
        temperature=temperature,         # 控制多樣性
        top_k=top_k,                     # 限制前 K 個機率最高 token
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    # 解碼完整句子
    full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 取最後補的部分（下一詞）
    added_text = full_text[len(input_text):]

    return added_text.strip(), full_text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/421M [00:00<?, ?B/s]

In [3]:
# ==== 測試 ====
input_text = "我今天想去"
next_word, full_sentence = predict_next_word(input_text)
print(f"原句：{input_text}")
print(f"補的詞：{next_word}")
print(f"完整句子：{full_sentence}")

原句：我今天想去
補的詞：想 去 乐
完整句子：我 今 天 想 去 乐


In [4]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
if torch.cuda.is_available():
    model.to("cuda")

def generate_next_word(text, max_new_tokens=3):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,  # 最多補 3 個 token（大概一個詞）
        do_sample=True,
        top_k=50,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    next_word = generated_text[len(text):].strip()
    return next_word

# ==== 互動式 ====
sentence = "我今天想去"
print(f"初始句子：{sentence}")

while True:
    suggestion = generate_next_word(sentence)
    print(f"建議補充詞：{suggestion}")
    choice = input("是否採用？(y/n/exit): ").strip().lower()
    if choice == "y":
        sentence += suggestion
        print(f"目前句子：{sentence}")
    elif choice == "exit":
        break
    else:
        print("重新生成...")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


初始句子：我今天想去
建議補充詞：想 去 里 面 吃
是否採用？(y/n/exit): y


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


目前句子：我今天想去想 去 里 面 吃
建議補充詞：里 面 吃 真 的 很
是否採用？(y/n/exit): exit


In [5]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

# 選擇 GPT-2 中文模型（更符合需求）
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

if torch.cuda.is_available():
    model.to("cuda")

def generate_next_phrase(text, max_new_tokens=5):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,  # 最多補 5 個 token
        do_sample=True,
        top_k=50,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    added_text = generated_text[len(text):].strip()

    # 控制補的字數（大約三個字，取前三個中文字）
    phrase = ''.join([ch for ch in added_text if ch.strip()])[:3]
    return phrase, generated_text

# ==== 測試 ====
sentence = "我今天想去"
phrase, full_sentence = generate_next_phrase(sentence)
print(f"原句：{sentence}")
print(f"補的短語：約三個字 → {phrase}")
print(f"完整句子：{full_sentence}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


原句：我今天想去
補的短語：約三個字 → 想去乐
完整句子：我 今 天 想 去 乐 ， 于 是 就


In [6]:
from transformers import pipeline, GPT2LMHeadModel, AutoTokenizer
import torch

# 使用中文 GPT-2
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)


if torch.cuda.is_available():
    model.to("cuda")

# 計算句子困惑度（Perplexity）
def calculate_perplexity(sentence):
    encodings = tokenizer(sentence, return_tensors='pt')
    if torch.cuda.is_available():
        encodings = {k: v.to("cuda") for k, v in encodings.items()}

    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()  # perplexity

# 生成候選詞並過濾
def suggest_next_word(sentence, max_new_tokens=3, num_return_sequences=5):
    results = generator(
        sentence,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        temperature=0.8,
        num_return_sequences=num_return_sequences
    )

    candidates = []
    for r in results:
        text = r['generated_text'][len(sentence):].strip()
        word = ''.join([ch for ch in text if ch.strip()])[:3]
        if word:
            candidates.append(word)

    # 根據困惑度排序（越小越通順）
    scored_candidates = []
    for word in candidates:
        new_sentence = sentence + word
        perplexity = calculate_perplexity(new_sentence)
        scored_candidates.append((word, perplexity))

    scored_candidates.sort(key=lambda x: x[1])
    return scored_candidates[:3]  # 取前3個最通順的詞

Device set to use cuda:0


In [7]:
sentence = "我今天想去"
print(f"初始句子：{sentence}")

while True:
    candidates = suggest_next_word(sentence)
    print("建議詞（已按語境一致性排序）:")
    for i, (word, score) in enumerate(candidates):
        print(f"{i}. {word} (PPL={score:.2f})")

    choice = input("選擇(輸入序號採用，n重新生成，exit結束): ").strip().lower()

    if choice == "exit":
        break
    elif choice.isdigit() and 0 <= int(choice) < len(candidates):
        sentence += candidates[int(choice)][0]
        print(f"目前句子：{sentence}")
    else:
        print("重新生成...")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


初始句子：我今天想去
建議詞（已按語境一致性排序）:
0. 百子湾 (PPL=81.31)
1. 七宝老 (PPL=88.28)
2. 上面写 (PPL=97.89)
選擇(輸入序號採用，n重新生成，exit結束): 0
目前句子：我今天想去百子湾
建議詞（已按語境一致性排序）:
0. 吃东西 (PPL=46.96)
1. 吃晚饭 (PPL=47.04)
2. 吃饭， (PPL=56.62)
選擇(輸入序號採用，n重新生成，exit結束): 0
目前句子：我今天想去百子湾吃东西
建議詞（已按語境一致性排序）:
0. ，结果 (PPL=35.65)
1. 我想吃 (PPL=48.18)
2. ，就来 (PPL=50.08)
選擇(輸入序號採用，n重新生成，exit結束): exit


In [10]:
from transformers import pipeline, GPT2LMHeadModel, BertTokenizerFast
import torch

# 使用繁體中文 GPT-2
model_name = "ckiplab/gpt2-base-chinese"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

if torch.cuda.is_available():
    model.to("cuda")

# 計算 Perplexity
def calculate_perplexity(text):
    encodings = tokenizer(text, return_tensors='pt')
    if torch.cuda.is_available():
        encodings = {k: v.to("cuda") for k, v in encodings.items()}
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    return torch.exp(outputs.loss).item()

# 產生候選詞並排序
def suggest_next_three(sentence, max_new_tokens=3, num_return_sequences=5):
    results = generator(
        sentence,
        max_new_tokens=max_new_tokens,  # 預測 3 個 token
        do_sample=True,
        top_k=50,
        temperature=0.8,
        num_return_sequences=num_return_sequences
    )

    candidates = []
    for r in results:
        text = r['generated_text'][len(sentence):].strip()
        # 只取前三個字
        phrase = ''.join([ch for ch in text if ch.strip()])[:3]
        if phrase:
            candidates.append(phrase)

    # 去重
    candidates = list(set(candidates))

    # 根據 perplexity 排序
    scored_candidates = []
    for phrase in candidates:
        new_sentence = sentence + phrase
        ppl = calculate_perplexity(new_sentence)
        scored_candidates.append((phrase, ppl))

    scored_candidates.sort(key=lambda x: x[1])  # 越小越好
    return scored_candidates[:3]  # 取前三候選

# ==== 互動式 ====
sentence = "我今天想去"
print(f"初始句子：{sentence}")

while True:
    candidates = suggest_next_three(sentence)
    print("候選詞（已按語境一致性排序）:")
    for i, (phrase, score) in enumerate(candidates):
        print(f"{i}. {phrase} (PPL={score:.2f})")

    choice = input("選擇(輸入序號採用，n重新生成，exit結束): ").strip().lower()

    if choice == "exit":
        break
    elif choice.isdigit() and 0 <= int(choice) < len(candidates):
        sentence += candidates[int(choice)][0]
        print(f"目前句子：{sentence}")
    else:
        print("重新生成...")


Device set to use cuda:0


初始句子：我今天想去
候選詞（已按語境一致性排序）:
0. 你的, (PPL=708.92)
1. 麼,我 (PPL=777.97)
2. 好,這 (PPL=830.82)
選擇(輸入序號採用，n重新生成，exit結束): 0
目前句子：我今天想去你的,
候選詞（已按語境一致性排序）:
0. 我今天 (PPL=211.59)
1. 有什麼 (PPL=238.24)
2. 我真的 (PPL=240.90)
選擇(輸入序號採用，n重新生成，exit結束): exit


In [11]:
from transformers import pipeline, GPT2LMHeadModel, BertTokenizerFast
import torch

# 載入繁體中文 GPT-2
model_name = "ckiplab/gpt2-base-chinese"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

if torch.cuda.is_available():
    model.to("cuda")

# 計算 Perplexity
def calculate_perplexity(text):
    encodings = tokenizer(text, return_tensors='pt')
    if torch.cuda.is_available():
        encodings = {k: v.to("cuda") for k, v in encodings.items()}
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    return torch.exp(outputs.loss).item()

# 產生候選詞（固定三個字）並排序
def suggest_next_phrase(sentence, num_return_sequences=5):
    results = generator(
        sentence,
        max_new_tokens=5,  # 預測一些字，然後截取前三字
        do_sample=True,
        top_k=50,
        temperature=0.8,
        num_return_sequences=num_return_sequences
    )

    candidates = []
    for r in results:
        added = r['generated_text'][len(sentence):].strip()
        phrase = ''.join([ch for ch in added if ch.strip()])[:3]  # 取前三個字
        if phrase:
            candidates.append(phrase)

    candidates = list(set(candidates))  # 去重

    # 用 PPL 排序
    scored = []
    for phrase in candidates:
        new_sentence = sentence + phrase
        ppl = calculate_perplexity(new_sentence)
        scored.append((phrase, ppl))

    scored.sort(key=lambda x: x[1])  # 越小越通順
    return scored[:3]  # 取前三名

# ==== 互動式 ====
sentence = "我今天想去"
print(f"初始句子：{sentence}")

while True:
    candidates = suggest_next_phrase(sentence)
    print("\n候選詞（語境一致性排序）:")
    for i, (phrase, score) in enumerate(candidates):
        print(f"{i}. {phrase} (PPL={score:.2f})")

    choice = input("選擇(輸入序號採用，n重新生成，exit結束): ").strip().lower()

    if choice == "exit":
        break
    elif choice.isdigit() and 0 <= int(choice) < len(candidates):
        sentence += candidates[int(choice)][0]
        print(f"目前句子：{sentence}")
    else:
        print("重新生成...")


Device set to use cuda:0


初始句子：我今天想去

候選詞（語境一致性排序）:
0. ,我覺 (PPL=383.39)
1. 一趟, (PPL=482.10)
2. 我家, (PPL=547.48)
選擇(輸入序號採用，n重新生成，exit結束): 1
目前句子：我今天想去一趟,

候選詞（語境一致性排序）:
0. 這不是 (PPL=167.21)
1. 不見得 (PPL=240.49)
2. 看我們 (PPL=242.69)
選擇(輸入序號採用，n重新生成，exit結束): 2
目前句子：我今天想去一趟,看我們

候選詞（語境一致性排序）:
0. 在台灣 (PPL=134.46)
1. 很了解 (PPL=177.61)
2. 人們有 (PPL=201.64)
選擇(輸入序號採用，n重新生成，exit結束): 0
目前句子：我今天想去一趟,看我們在台灣

候選詞（語境一致性排序）:
0. 有什麼 (PPL=86.21)
1. 的歷史 (PPL=97.25)
2. 生活, (PPL=98.85)
選擇(輸入序號採用，n重新生成，exit結束): 0
目前句子：我今天想去一趟,看我們在台灣有什麼

候選詞（語境一致性排序）:
0. 不敢去 (PPL=77.92)
1. 的活動 (PPL=79.02)
2. 力。」 (PPL=81.31)
選擇(輸入序號採用，n重新生成，exit結束): exit
